In [23]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from quantopian.pipeline import Pipeline
from quantopian.pipeline.data.builtin import USEquityPricing
from quantopian.research import run_pipeline
from quantopian.pipeline.data import Fundamentals
from quantopian.pipeline.factors import CustomFactor,Returns,Latest,DailyReturns
from quantopian.pipeline.classifiers import Classifier
from quantopian.pipeline.filters import QTradableStocksUS,Q1500US
from quantopian.pipeline.classifiers.fundamentals import Sector
import itertools
from scipy.stats import skew

###Very Significant Factor:
1.BP_Rank + Cap_Rank - Sortino_Ratio;
  start_date ='2016-01-01',end_date = '2017-12-31'
  start_date ='2009-01-01',end_date = '2012-01-01' #Very Strong Quintile 2

In [24]:
MORNINGSTAR_SECTOR_CODES = {
     -1: 'Misc',
    101: 'Basic Materials',
    102: 'Consumer Cyclical',
    103: 'Financial Services',
    104: 'Real Estate',
    205: 'Consumer Defensive',
    206: 'Healthcare',
    207: 'Utilities',
    308: 'Communication Services',
    309: 'Energy',
    310: 'Industrials',
    311: 'Technology' ,    
}

### Custom Factors

In [25]:
class Momentum(CustomFactor):
    # get the return from last month
    inputs = [Returns(window_length = 20)]
    window_length = 20
    
    def compute(self,today,assets,out,lag_returns):
        out[:] = lag_returns[0]

In [26]:
class SortinoRatio(CustomFactor):
    inputs = [Returns(window_length = 2 )]
    window_length = 20
    def compute(self, today, asset_ids, out, returns):
        out[:]=returns.mean(axis = 0)/returns[returns < 0].std(axis=0)

In [27]:
class Skewness():
    inputs = [Returns(window_length = 2 )]
    window_length = 20
    def compute(self, today, asset_ids, out, returns):
        out[:]=skew(returns)

### Make Pipeline

##### Pipeline for CAP,BP,MOM

In [28]:
def make_pipeline():
    pipe = Pipeline()

    returns = Returns(inputs = [USEquityPricing.close],window_length=20)
    worst_returns_filter = returns.percentile_between(0,20)
    
    returns = Returns(window_length = 2)
    universe = Q1500US() & worst_returns_filter
    pipe = Pipeline(
        columns = {
            'Returns':returns,
            'sector':Sector(),
            'Sortino_Ratio':SortinoRatio()
        },
        screen = universe
    )
    return pipe

### Run Pipeline

In [34]:
pipe = make_pipeline()
results = run_pipeline(pipe,'2006-01-01','2012-01-01')

In [35]:
results.head(7)

Returns  Sortino_Ratio  sector
2006-01-03 00:00:00+00:00 Equity(60 [ABS])   -0.000468      -0.245557     205
                          Equity(122 [ADI])  -0.009666      -0.228951     311
                          Equity(209 [AM])   -0.014798      -0.434690     102
                          Equity(553 [ASEI]) -0.009821      -0.231660     310
                          Equity(583 [AT])   -0.002689      -0.120167     308
                          Equity(607 [ATML]) -0.012862      -0.314447     311
                          Equity(663 [AVY])  -0.013367      -0.165962     310

Data Cleaning: Sortino Ratio

In [36]:
Average_Sortino_Ratio = results['Sortino_Ratio'].groupby(level = 0).mean()
Std_Sortino_Ratio = results['Sortino_Ratio'].groupby(level = 0).std()

CREATE DATA COLUMNS

In [37]:
data = results[['Returns']].set_index(results.index)
asset_list_sizes = [group[1].size for group in data.groupby(level = 0 )]

DATA COLUMN : Sortino Ratio

In [38]:
data['Sortino_Ratios'] = results['Sortino_Ratio']
Avg_Sortino_Ratio_column = [[idx] * size for idx, size in zip(Average_Sortino_Ratio,asset_list_sizes)]
data['Average_Sortino_Ratio'] = list(itertools.chain(*Avg_Sortino_Ratio_column))
Std_Sortino_Ratio_column = [[idx] * size for idx, size in zip(Std_Sortino_Ratio,asset_list_sizes)]
data['Std_Sortino_Ratio'] = list(itertools.chain(*Std_Sortino_Ratio_column))
data['Normal_Sortino_Ratio'] = (data['Sortino_Ratios']-data['Average_Sortino_Ratio'])/data['Std_Sortino_Ratio']

#### Construct A Factor

1.Only Sortino

In [49]:
FACTOR = -data['Normal_Sortino_Ratio']
sectors = results['sector']
asset_list = results.index.levels[1]
prices = get_pricing(asset_list,start_date ='2006-01-01',end_date = '2012-01-01',fields = 'close_price')
period = (1,5,10,30)

In [50]:
import alphalens as al

In [51]:
factor_data =  al.utils.get_clean_factor_and_forward_returns(factor = FACTOR,
                                                           prices = prices,#Pass Price by DataFrame
                                                           groupby = sectors,#Group by Dictonary
                                                           groupby_labels = MORNINGSTAR_SECTOR_CODES,
                                                           periods = period,
                                                           quantiles = 20
                                                           )

Dropped 2.2% entries from factor data: 2.2% in forward returns computation and 0.0% in binning phase (set max_loss=0 to see potentially suppressed Exceptions).
max_loss is 35.0%, not exceeded: OK!


In [ ]:
al.tears.create_full_tear_sheet(factor_data,by_group = True)

Quantiles Statistics


,min,max,mean,std,count,count %
factor_quantile,,,,,,
1,-11.126132,-0.486751,-1.330801,0.521120,20257,5.178249
2,-1.576714,-0.351818,-0.945116,0.122131,19543,4.995731
3,-1.226658,-0.306174,-0.821436,0.091009,19439,4.969146
4,-0.988504,-0.248471,-0.730867,0.077238,19611,5.013114
5,-0.917477,-0.215839,-0.652615,0.069340,19504,4.985762
6,-0.826609,-0.179367,-0.580568,0.063660,19387,4.955853
7,-0.746493,-0.159913,-0.509331,0.059826,19448,4.971446
8,-0.660171,-0.070908,-0.435901,0.057211,19600,5.010302
9,-0.520129,0.002222,-0.359630,0.055955,19368,4.950996


Returns Analysis


,1D,5D,10D,30D
Ann. alpha,0.067,0.031,0.013,0.007
beta,0.134,0.155,0.146,0.212
Mean Period Wise Return Top Quantile (bps),8.242,6.717,4.371,2.940
Mean Period Wise Return Bottom Quantile (bps),-1.410,-0.213,-0.235,-0.023
Mean Period Wise Spread (bps),9.652,6.035,3.631,1.935
